# **Impor Pustaka**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

# **Baca Dataset**

In [ ]:
df = pd.read_csv('data.csv')
df.head()

,gameId,gameDuration,hasWon,frame,goldDiff,expDiff,champLevelDiff,isFirstTower,isFirstBlood,killedFireDrake,killedWaterDrake,killedAirDrake,killedEarthDrake,killedElderDrake,lostFireDrake,lostWaterDrake,lostAirDrake,lostEarthDrake,lostElderDrake,killedBaronNashor,lostBaronNashor,killedRiftHerald,lostRiftHerald,destroyedTopInhibitor,destroyedMidInhibitor,destroyedBotInhibitor,lostTopInhibitor,lostMidInhibitor,lostBotInhibitor,destroyedTopNexusTurret,destroyedMidNexusTurret,destroyedBotNexusTurret,lostTopNexusTurret,lostMidNexusTurret,lostBotNexusTurret,destroyedTopBaseTurret,destroyedMidBaseTurret,destroyedBotBaseTurret,lostTopBaseTurret,lostMidBaseTurret,lostBotBaseTurret,destroyedTopInnerTurret,destroyedMidInnerTurret,destroyedBotInnerTurret,lostTopInnerTurret,lostMidInnerTurret,lostBotInnerTurret,destroyedTopOuterTurret,destroyedMidOuterTurret,destroyedBotOuterTurret,lostTopOuterTurret,lostMidOuterTurret,lostBotOuterTurret,kills,deaths,assists,wardsPlaced,wardsDestroyed,wardsLost
0,4546233126,1443000,1,10,-448,-147,-0.2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,7,5,21,3,5
1,4546233126,1443000,1,12,-1306,-925,-0.6,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,11,6,28,4,6
2,4546233126,1443000,1,14,2115,2578,0.4,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,10,11,12,35,4,6
3,4546233126,1443000,1,16,1195,2134,0.4,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,10,12,12,45,6,10
4,4546233126,1443000,1,18,2931,4382,0.6,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,13,13,16,49,7,12


# **Eksplorasi**

Mengecek jumlah game yang ada

In [ ]:
game_ids = df['gameId'].unique()
print('Ada ' + str(len(game_ids)) + ' gameId')

Ada 24912 gameId


## Melakukan pendekatan OOP dengan membuat Kelas

Membuat kelas Dataset

In [ ]:
class Dataset:
  def __init__(self, has_won_count, has_lost_count):
    self.has_won_count = has_won_count
    self.has_lost_count = has_lost_count

  def display(self):
    print(f'Menang sejumlah {self.has_won_count} ({self.__get_won_percentage()}%)')
    print(f'Kalah sejumlah {self.has_lost_count} ({self.__get_lost_percentage()}%)')

  
  def __get_won_percentage(self):
    return self.has_won_count / (self.has_won_count + self.has_lost_count)


  def __get_lost_percentage(self):
    return self.has_lost_count / (self.has_won_count + self.has_lost_count)


Membuat kelas Feature

In [ ]:
class Feature:
  def __init__(self, name):
    self.name = name

Membuat kelas untuk fitur yang hanya terkait dengan 1 kolom

In [ ]:
class SingleFeature(Feature):
  def __init__(self, field, name):
    super().__init__(name)
    self.field = field


In [ ]:
class SingleBooleanFeature(SingleFeature):
  def explore(self):
    won_true_percentages = []
    lost_true_percentages = []

    for game_id in game_ids:
      games = df[df.gameId == game_id]
      series = games[self.field]
      true_count = float(len(series[series == 1]))

      if games.iloc[0].hasWon == 1:
        won_true_percentages.append(true_count / float(len(series)) * 100)

      else:
        lost_true_percentages.append(true_count / float(len(series)) * 100)

    
    if len(won_true_percentages) == 0: won_true_percentages.append(0)
    if len(lost_true_percentages) == 0: lost_true_percentages.append(0)

    self.won_true_avg_percentage = sum(won_true_percentages) / len(won_true_percentages)
    self.lost_true_avg_percentage = sum(lost_true_percentages) / len(lost_true_percentages)


  def display(self):
    print(f'In won game, True percentage of {self.name} feature is {self.won_true_avg_percentage}')
    print(f'In lost game, True percentage of {self.name} feature is {self.lost_true_avg_percentage}')

Membuat kelas untuk fitur yang terkait dengan beberapa kolom boolean

In [ ]:
class MultipleBooleanFeature(Feature):
  def __init__(self, fields, name):
    super().__init__(name)
    self.fields = fields

  
  def explore(self):
    won_true_avg_percentages = []
    lost_true_avg_percentages = []

    for field in self.fields:
      feature = SingleBooleanFeature(field, self.name)
      feature.explore()
      
      won_true_avg_percentages.append(feature.won_true_avg_percentage)
      lost_true_avg_percentages.append(feature.lost_true_avg_percentage)

    if len(won_true_avg_percentages) == 0: won_true_avg_percentages.append(0)
    if len(lost_true_avg_percentages) == 0: lost_true_avg_percentages.append(0)

    won_true_avg_percentage = sum(won_true_avg_percentages) / len(won_true_avg_percentages)
    lost_true_avg_percentage = sum(lost_true_avg_percentages) / len(lost_true_avg_percentages)

    print(f'In won game, True percentage of {self.name} features is {won_true_avg_percentage}')
    print(f'In lost game, True percentage of {self.name} features is {lost_true_avg_percentage}')

Membuat kelas untuk numerical feature yang terkait dengan 1 kolom

In [ ]:
class NumericalFeature(SingleFeature):
  def explore(self):
    has_won_total_last_value = 0
    has_won_total_value = 0
    has_lost_total_last_value = 0
    has_lost_total_value = 0

    for game_id in game_ids:
      games = df[df.gameId == game_id]
      if (len(games) < 1): continue
      lastGame = games.iloc[len(games) - 1]

      # Check whether has won
      if games.iloc[0].hasWon == 1:
        has_won_total_last_value += lastGame[self.field]
        has_won_total_value += games[self.field].sum() / len(games)

      else:
        has_lost_total_last_value += lastGame[self.field]
        has_lost_total_value += games[self.field].sum() / len(games)

    
    self.won_average_last_value = has_won_total_last_value / dataset.has_won_count
    self.lost_average_last_value = has_won_total_value / dataset.has_won_count
    
    self.won_average_value = has_lost_total_last_value / dataset.has_lost_count
    self.lost_average_value = has_lost_total_value / dataset.has_lost_count

    self.__display()


  def __display(self):
    print(f'Average last {self.name} of hasWon game: {self.won_average_last_value}')
    print(f'Average {self.name} of hasWon game: {self.won_average_value}\n')

    print(f'Average last {self.name} of hasLost game: {self.lost_average_last_value}')
    print(f'Average {self.name} of hasLost game: {self.lost_average_value}')

## Membuat object dataset

In [ ]:
dataset = Dataset(0, 0)

for game_id in game_ids:
  games = df[df.gameId == game_id]
  if (len(games) < 1): continue

  # Check whether has won
  if games.iloc[0].hasWon == 1:
    dataset.has_won_count += 1

  else:
    dataset.has_lost_count += 1


dataset.display()

Menang sejumlah 12539 (0.5033317276814386%)
Kalah sejumlah 12373 (0.4966682723185613%)


## Membuat object-object fitur pilihan dan mengeksplorasinya

Eksplorasi fitur goldDiff

In [ ]:
goldDiff_feature = NumericalFeature('goldDiff', 'Gold Difference')
goldDiff_feature.explore()

Average last Gold Difference of hasWon game: 8414.75683866337
Average Gold Difference of hasWon game: -8648.029014790269

Average last Gold Difference of hasLost game: 3903.847958203456
Average Gold Difference of hasLost game: -3883.7115219344087


Eksplorasi fitur expDiff

In [ ]:
expDiff_feature = NumericalFeature("expDiff", "EXP Difference")
expDiff_feature.explore()


Average last EXP Difference of hasWon game: 8307.144899904672
Average EXP Difference of hasWon game: 3232.1301890568748

Average last EXP Difference of hasLost game: -9195.359194395796
Average EXP Difference of hasLost game: -3596.6993686954106


In [ ]:
Eksplorasi fitur champLevelDiff

In [ ]:
champLevelDiff_feature = NumericalFeature('champLevelDiff', 'Championship Level Difference')
champLevelDiff_feature.explore()

Average last Championship Level Difference of hasWon game: 1.104250737698379
Average Championship Level Difference of hasWon game: -1.1904146124626174

Average last Championship Level Difference of hasLost game: 0.5128895778129525
Average Championship Level Difference of hasLost game: -0.5567144860819065


Eksplorasi fitur isFirstTower

In [ ]:
isFirstTower_feature = SingleBooleanFeature('isFirstTower', 'Is First Tower')
isFirstTower_feature.explore()
isFirstTower_feature.display()

In won game, True percentage of Is First Tower feature is 70.44112289619368
In lost game, True percentage of Is First Tower feature is 45.77988016445224


Eksplorasi fitur isFirstBlood

In [ ]:
isFirstBlood_feature = SingleBooleanFeature('isFirstBlood', 'Is First Blood')
isFirstBlood_feature.explore()
isFirstBlood_feature.display()

In won game, True percentage of Is First Blood feature is 99.96466755442282
In lost game, True percentage of Is First Blood feature is 99.55610469464754


Eksplorasi fitur killedFireDrake

In [ ]:
killedFireDrake_feature = SingleBooleanFeature('killedFireDrake', 'Killed Fire Drake')
killedFireDrake_feature.explore()
killedFireDrake_feature.display()

In won game, True percentage of Killed Fire Drake feature is 32.49482586594573
In lost game, True percentage of Killed Fire Drake feature is 16.142218643644256


Eksplorasi fitur-fitur terkait killed

In [ ]:
killed_multiple_features = MultipleBooleanFeature(
    ['killedFireDrake', 'killedWaterDrake', 'killedAirDrake', 'killedEarthDrake', 'killedElderDrake', 'killedBaronNashor', 'killedRiftHerald'],
    'Killed'
)
killed_multiple_features.explore()

In won game, True percentage of Killed features is 26.538756902330338
In lost game, True percentage of Killed features is 14.443511340973373


Ekplorasi fitur destroyedTopInhibitor

In [ ]:
destroyedTopInhibitor_feature = SingleBooleanFeature('destroyedTopInhibitor', 'Destroyted Top Inhibitor')
destroyedTopInhibitor_feature.explore()
destroyedTopInhibitor_feature.display()

In won game, True percentage of Destroyted Top Inhibitor feature is 3.2276715238123272
In lost game, True percentage of Destroyted Top Inhibitor feature is 0.44492294982851793


Eksplorasi fitur-fitur terkait destroyed

In [ ]:
destroyed_multiple_features = MultipleBooleanFeature(
    ['destroyedTopInhibitor', 'destroyedMidInhibitor', 'destroyedBotInhibitor', 'destroyedTopNexusTurret', 'destroyedMidNexusTurret', 'destroyedBotNexusTurret', 'destroyedTopBaseTurret', 'destroyedMidBaseTurret', 'destroyedBotBaseTurret', 'destroyedTopInnerTurret', 'destroyedMidInnerTurret', 'destroyedBotInnerTurret', 'destroyedTopOuterTurret', 'destroyedMidOuterTurret', 'destroyedBotOuterTurret'],
    'Destroyed'
)
destroyed_multiple_features.explore()

In won game, True percentage of Destroyed features is 16.504507629379845
In lost game, True percentage of Destroyed features is 6.668972835346449


Eksplorasi fitur kills

In [ ]:
kills_feature = NumericalFeature('kills', 'Kills')
kills_feature.explore()

Average last Kills of hasWon game: 30.718478347555628
Average Kills of hasWon game: 20.151943748484605

Average last Kills of hasLost game: 17.822881287866473
Average Kills of hasLost game: 12.831136546880538


Eksplorasi fitur deaths

In [ ]:
deaths_feature = NumericalFeature('deaths', 'Deaths')
deaths_feature.explore()

Average last Deaths of hasWon game: 19.70037483052875
Average Deaths of hasWon game: 31.13634526792209

Average last Deaths of hasLost game: 12.541613714481288
Average Deaths of hasLost game: 17.973817671009623


Eksplorasi fitur assists

In [ ]:
assists_feature = NumericalFeature('assists', 'Assists')
assists_feature.explore()

Average last Assists of hasWon game: 44.73562485046654
Average Assists of hasWon game: 28.85411783722622

Average last Assists of hasLost game: 22.377160624424352
Average Assists of hasLost game: 16.25046406176805
